In [1]:
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import tqdm
import numpy as np

In [2]:
#In Python: Define a database name (we're using a dataset on births, so I call it 
# birth_db), and your username for your computer (CHANGE IT BELOW). 
dbname = 'music_db'
username = 'shayneufeld'

In [3]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://shayneufeld@localhost/music_db


In [4]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


# create album supporter and album info tables fromd bandcamp album scraper

In [6]:
info_folder = '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_info'
supporter_folder = '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_supporters'
files_info = os.listdir(info_folder)
files_supporter = os.listdir(supporter_folder)
info_files_stored = []

In [7]:
info_master = pd.DataFrame()
for info in tqdm.tqdm(files_info):
    temp_info = pd.read_csv(os.path.join(info_folder,info),index_col = 0)
    info_master = info_master.append(temp_info)

100%|██████████| 3140/3140 [01:16<00:00, 16.68it/s] 


In [8]:
info_master.index = np.arange(info_master.shape[0])

In [10]:
info_master.columns.values

array(['"Arch Echo" Debut Compact Disc', '"Gire" Jewel Case CD',
       '"Wormhole Edition" CD',
       "'Kumiko, the Treasure Hunter' Original Soundtrack by the Octopus Project LIMITED EDITION RED SHELL CASSETTE!",
       "'THICK' Double 90 Minute Cassette Version",
       '\'WORKED ON\' LIMITED 11x17" GLITCH ART LITHOGRAPH POSTER',
       "'WORKED ON' LIVE CASSETTE MIXTAPE",
       "'Yes! I need.' Handmade Limited Cassette Version",
       '(IN)DEFINITE LYFE CASSETTE VERSION',
       '(SALE) Outcasts (COLLECTION) (4 CD)',
       '***PRE-ORDER*** 12" Vinyl LP (Translucent Gold)', '10 CD Bundle',
       '10 CD Sabled Sun Bundle', '12" Black Vinyl Repress',
       '12" Double LP', '12" EP', '12" Future Blue Vinyl [84V1]',
       '12" Gatefold LP + Booklet', '12" Gatefold LP + Poster',
       '12" Gatefold MLP', '12" LP (Black 180g)', '12" LP (Opaque Purple)',
       '12" Limited Edition White Vinyl', '12" MLP',
       '12" Red Vinyl [126V1]', '12" Turquoise Vinyl [23XV1]',
       '12" V

In [11]:
col_dict = {}
df = pd.DataFrame(columns=['Bundle','Vinyl','CD','Tape','Digital','Unclassified'])
df['album_name'] = info_master.album_name
df['artist_name'] = info_master.artist_name
df['location'] = info_master.location
df['tags'] = info_master.tags
df['year'] = info_master.year

In [41]:
info_master.loc[info_master.year.isnull(),'year'] = 1900

In [48]:
for i in tqdm.tqdm(np.arange(info_master.shape[0])):
    curr = info_master.iloc[i]
    a = curr[~pd.isnull(curr)]
    cols = a.index
    cols = cols.drop(['album_name','artist_name','location','tags','year'])
    
    for col in cols:
        if (('Bundle' in col) or ('Discography' in col) or ('bundle' in col)):
            #print('Bundle: %s' % col)
            #col_dict.update({col:'Bundle'})
            df.loc[i,'Bundle'] = info_master.loc[i,col]
                   
        elif (('Vinyl' in col) or ('12"' in col) or ('vinyl' in col) or ('LP' in col)):
            #print('Vinyl: %s' % col)
            #col_dict.update({col:'Vinyl'})
            df.loc[i,'Vinyl'] = info_master.loc[i,col]
                   
        elif (('Compact' in col) or ('CD' in col)):
            #print('CD: %s' % col)
            #col_dict.update({col:'CD'})
            df.loc[i,'CD'] = info_master.loc[i,col]
                   
        elif(('Cassette' in col) or ('Tape' in col) or ('CASSETTE' in col)):
            #print('Tape: %s' % col)
            #col_dict.update({col:'Tape'})
            df.loc[i,'Tape'] = info_master.loc[i,col]
                   
        elif(('Digital' in col) or ('Download' in col)):
            #print('Digital: %s' % col)
            #col_dict.update({col:'Digital'})
            df.loc[i,'Digital'] = info_master.loc[i,col]
                   
        else:
            #print('Unclassified: %s' % col)
            #col_dict.update({col:'Unclassified'})
            df.loc[i,'Unclassified'] = info_master.loc[i,col]
                   

100%|██████████| 3139/3139 [00:07<00:00, 412.03it/s]


In [43]:
df.shape

(3139, 11)

In [44]:
info_master.shape

(3139, 557)

In [60]:
df.head(5)

,Bundle,Vinyl,CD,Tape,Digital,Unclassified,album_name,artist_name,location,tags,year
0,NaN,NaN,NaN,NaN,$1,NaN,"""Condos""-LiveatTheBellHouse",Welcome to Night Vale,New York,['podcasts'],2014.0
1,(,NaN,NaN,NaN,70.4,NaN,"""Emerson""UnreleasedDemo",Wintergatan,Gothenburg,['pop'],2017.0
2,NaN,NaN,NaN,NaN,$1,NaN,"""TheDebate""-LiveatRoulette",Welcome to Night Vale,New York,"['podcasts', 'night vale']",2014.0
3,NaN,NaN,NaN,NaN,$5,NaN,"""TheInvestigators""Live",Welcome to Night Vale,New York,"['podcasts', 'cecil baldwin', 'comedy', 'night...",2016.0
4,NaN,NaN,NaN,NaN,$1,NaN,"""TheLibrarian""-LiveatSkirball",Welcome to Night Vale,New York,"['night vale', 'podcasts', 'welcome to night v...",2015.0


In [61]:
#insert into db
df.to_sql('album_info',engine,if_exists='replace')

In [50]:
supporter_folder = '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_supporters'
files_supp = os.listdir(supporter_folder)
supp_files_stored = []

# Add supporters files to DB

In [51]:
for supp in tqdm.tqdm(files_supporter):
    STORED = supp in supp_files_stored

    if not STORED:
        temp_supp = pd.read_csv(os.path.join(supporter_folder,supp),index_col=0)
        temp_supp.to_sql('album_supporters',engine,if_exists='append')
        supp_files_stored.append(supp)

100%|██████████| 3140/3140 [03:07<00:00, 16.76it/s]


In [58]:
#change supp_files_stored to pandas so I can save it
files_stored_df = pd.DataFrame(data={'files_stored':supp_files_stored})
files_stored_df.to_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/supp_files_stored_0731.csv')

# Create spotify table

In [55]:
pwd

'/Users/shayneufeld/GitHub/pricecamp/notebooks'

In [256]:
spotify_df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/spotify_df.csv',index_col=0)
spotify_df.to_sql('spotify_table', engine, if_exists='replace')

# Create bandcamp table

In [8]:
cd '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed/'

/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed


In [9]:
folder = '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed'
files = os.listdir(folder)

for file in files:
    curr_df = pd.read_csv(file,index_col=0)
    if not len(curr_df) == 0:
        if curr_df.columns[0] == 'Unnamed: 0.1':
            curr_df = curr_df.drop('Unnamed: 0.1',axis=1)
        curr_df.to_sql('bandcamp_table',engine,if_exists='append')

# Create sales table

In [10]:
sales_df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/sales.csv',index_col=0)
sales_df.to_sql('sales_table', engine, if_exists='replace')

In [11]:
sales_df.shape

(74073, 22)

# Query from database

In [1]:
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM spotify_table WHERE match='True';
"""
sp_data_matches = pd.read_sql_query(sql_query,con)

sp_data_matches.shape[0]

NameError: name 'psycopg2' is not defined

In [262]:
sql_query = """
SELECT * FROM bandcamp_table;
"""
bandcamp_table = pd.read_sql_query(sql_query,con)

In [263]:
bandcamp_table.shape[0]

582344